## Bell State Preparation from Cirq

In [1]:
import cirq
from jaqalpaq.transpilers.cirq import jaqal_circuit_from_cirq_circuit
from jaqalpaq.generator import generate_jaqal_program

In [2]:
cirq_bell = cirq.Circuit()
qubits = [cirq.LineQubit(0), cirq.LineQubit(1)]
cirq_bell.append(cirq.H.on(qubits[0]))
cirq_bell.append(cirq.CNOT(*qubits))
print(cirq_bell)

0: ───H───@───
          │
1: ───────X───


In [3]:
cirq_ion_bell = cirq.ConvertToIonGates().convert_circuit(cirq_bell)
print(cirq_ion_bell)

0: ───PhX(1)───────────MS(0.25π)───PhX(-0.5)^0.5───S^-1───
                       │
1: ────────────────────MS(0.25π)───PhX(1)^0.5─────────────


In [4]:
jaqal_bell = jaqal_circuit_from_cirq_circuit(cirq_ion_bell)
print(generate_jaqal_program(jaqal_bell))

register allqubits[2]

prepare_all
R allqubits[0] 3.141592653589793 3.141592653589793
MS allqubits[0] allqubits[1] 0 1.5707963267948966
<
	R allqubits[0] -1.5707963267948972 1.5707963267948966
	R allqubits[1] 3.141592653589793 1.5707963267948966
>
Rz allqubits[0] -1.5707963267948966
measure_all



## Bell State Preparation from Quil

In [5]:
import pyquil
from pyquil.gates import *
from numpy import pi
from jaqalpaq.transpilers.quil import get_ion_qc, quil_gates

In [6]:
qg = quil_gates()
MS = qg["MS"]
R = qg["R"]
quil_bell = pyquil.Program()
quil_bell += X(0)
quil_bell += MS(0, pi/2, 0, 1)
quil_bell += R(-pi/2, pi/2, 0)
quil_bell += R(pi, pi/2, 1)
quil_bell += RZ(-pi/2, 0)
print(quil_bell)

X 0
MS(0,pi/2) 0 1
R(-pi/2,pi/2) 0
R(pi,pi/2) 1
RZ(-pi/2) 0



In [7]:
quil_qc = get_ion_qc(2)
jaqal_bell = quil_qc.compile(quil_bell)
print(generate_jaqal_program(jaqal_bell))

register qreg[2]

{
	prepare_all
	Px qreg[0]
	MS qreg[0] qreg[1] 0.0 1.5707963267948966
	R qreg[0] -1.5707963267948966 1.5707963267948966
	R qreg[1] 3.141592653589793 1.5707963267948966
	Rz qreg[0] -1.5707963267948966
	measure_all
}



## Bell State Preparation from ProjectQ

In [8]:
import projectq
from projectq.cengines import MainEngine, DummyEngine
from projectq.ops import H, CNOT, Measure, All
from jaqalpaq.transpilers.projectq import get_engine_list, JaqalBackend

In [9]:
backend = JaqalBackend()
engine_list = get_engine_list()
engine = MainEngine(backend, engine_list, verbose=True)
q1 = engine.allocate_qubit()
q2 = engine.allocate_qubit()
H | q1
CNOT | (q1, q2)
All(Measure) | [q1, q2]
engine.flush()
print(generate_jaqal_program(backend.circuit))

register q[2]

{
	prepare_all
	R q[0] 0 3.14159265359
	R q[0] 1.5707963267948966 3.141592653589413
	R q[0] 0 4.712388980384414
	R q[1] 0 1.570796326795
	MS q[0] q[1] 0 1.570796326795
	R q[0] 1.5707963267948966 1.570796326795
	measure_all
}



## The Automated Scheduler

In [10]:
from jaqalpaq.scheduler import schedule_circuit

In [11]:
scheduled_circuit = schedule_circuit(backend.circuit)
print(generate_jaqal_program(scheduled_circuit))

register q[2]

{
	prepare_all
	<
		R q[0] 0 3.14159265359
		R q[1] 0 1.570796326795
	>
	R q[0] 1.5707963267948966 3.141592653589413
	R q[0] 0 4.712388980384414
	MS q[0] q[1] 0 1.570796326795
	R q[0] 1.5707963267948966 1.570796326795
	measure_all
}



## Detailed Example: VQE Transpilation from Qiskit

### Constructing the VQE Circuit in Qiskit

In [12]:
import qiskit
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.transpiler import PassManager
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator

from jaqalpaq.transpilers.qiskit import jaqal_circuit_from_qiskit_circuit, ion_pass_manager, get_ion_instance
from jaqalpaq.emulator import run_jaqal_circuit
from qscout.v1.std import NATIVE_GATES

/usr/local/lib/python3.7/site-packages/qiskit/chemistry/__init__.py:170: DeprecationWarning: The package qiskit.chemistry is deprecated. It was moved/refactored to qiskit_nature (pip install qiskit-nature). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('chemistry', 'qiskit_nature', 'qiskit-nature')


In [13]:
atom_positions = [[0.0, 0.0, 0.0], [1.0, 0.0, 0.0]]

molecule = PySCFDriver(
    atom=''.join(["H %f %f %f; " % tuple(a) for a in atom_positions]),
    unit=UnitsType.BOHR,
    charge=0,
    spin=0,
    basis='sto6g'
).run()

fermionic_op = FermionicOperator(molecule.one_body_integrals, molecule.two_body_integrals)
qubit_op = fermionic_op.mapping(map_type='parity', threshold=0.00000001)
hfs = HartreeFock(num_orbitals=4, num_particles=2, qubit_mapping='parity', two_qubit_reduction=True)

uccsd = UCCSD(
    num_orbitals=4,
    num_particles=2,
    initial_state=hfs,
    qubit_mapping="parity",
    two_qubit_reduction=True,
    num_time_slices=1
)

params = (4.881817576986406e-05, -1.5180312771430236e-05, -0.07605047960428524)
superconducting_circuit = uccsd.construct_circuit(params).decompose()
print(superconducting_circuit)

/usr/local/lib/python3.7/site-packages/qiskit/chemistry/fermionic_operator.py:386: DeprecationWarning: The package qiskit.aqua.operators is deprecated. It was moved/refactored to qiskit.opflow (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  pauli_list = WeightedPauliOperator(paulis=[])
/usr/local/lib/python3.7/site-packages/qiskit/chemistry/fermionic_operator.py:394: DeprecationWarning: The variable qiskit.aqua.aqua_globals is deprecated. It was moved/refactored to qiskit.utils.algorithm_globals (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  task_args=(threshold,), num_processes=aqua_globals.num_processes)
/usr/local/lib/python3.7/site-packages/qiskit/chemistry/components/initial_states/hartree_fock.py:61: DeprecationWarning: The HartreeFock class is deprecated as of Aqua 0.9 and will be removed no earlier than 3 months a

     ┌───────────┐┌──────────────┐┌──────────────┐┌──────────────┐
q_0: ┤ U3(π,0,π) ├┤0             ├┤0             ├┤0             ├
     └───────────┘│  Evolution^1 ││  Evolution^1 ││  Evolution^1 │
q_1: ─────────────┤1             ├┤1             ├┤1             ├
                  └──────────────┘└──────────────┘└──────────────┘


/usr/local/lib/python3.7/site-packages/qiskit/chemistry/components/variational_forms/uccsd.py:429: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  qbits[i] = circuit.qubits[qbits[i].index]
/usr/local/lib/python3.7/site-packages/qiskit/chemistry/components/variational_forms/uccsd.py:429: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  qbits[i] = circuit.qubits[qbits[i].index]
/usr/local/lib/python3.7/site-packages/qiskit/chemistry/components/variational_forms/uccsd.py:429: DeprecationWarning: Back-references to from Bit instances to their containing Registers have been deprecated. Instead, inspect Registers to find their contained Bits.
  qbits[i] = circuit.qubits[qbits[i].index]
/usr/local/lib/python3.7/site-packages/qiskit/chemistry/components/var

### Unrolling to Ion Gates

In [14]:
ion_circuit = ion_pass_manager().run(superconducting_circuit)
print(ion_circuit)

global phase: -6.3998e-05
      ┌───────┐        ┌────┐       ┌────────┐            ┌────┐           »
q_0: ─┤ RZ(π) ├────────┤ √X ├───────┤ RZ(2π) ├────────────┤ √X ├───────────»
     ┌┴───────┴┐┌──────┴────┴─────┐┌┴────────┴┐┌──────────┴────┴──────────┐»
q_1: ┤ RZ(π/2) ├┤ JAQALR(π/2,π/2) ├┤ RZ(-π/2) ├┤ RZ(-3.03606255428605e-5) ├»
     └─────────┘└─────────────────┘└──────────┘└──────────────────────────┘»
«      ┌────────┐    ┌─────────┐     ┌─────────────────┐┌──────────┐»
«q_0: ─┤ RZ(3π) ├────┤ RZ(π/2) ├─────┤ JAQALR(π/2,π/2) ├┤ RZ(-π/2) ├»
«     ┌┴────────┤┌───┴─────────┴────┐└───┬──────────┬──┘└──┬───┬───┘»
«q_1: ┤ RZ(π/2) ├┤ JAQALR(π/2,-π/2) ├────┤ RZ(-π/2) ├──────┤ Z ├────»
«     └─────────┘└──────────────────┘    └──────────┘      └───┘    »
«     ┌──────────────────────────┐┌─────────┐┌──────────────────┐┌──────────┐»
«q_0: ┤ RZ(-9.76363515397281e-5) ├┤ RZ(π/2) ├┤ JAQALR(π/2,-π/2) ├┤ RZ(-π/2) ├»
«     └──────────┬────┬──────────┘└─────────┘└──────────────────┘└──────────┘»
«q

### Transpiling to Jaqal

In [15]:
jaqal_circuit = jaqal_circuit_from_qiskit_circuit(ion_circuit)
print(generate_jaqal_program(jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	Rz q[0] 3.141592653589793
	Sx q[0]
	Rz q[0] 6.283185307179586
	Sx q[0]
	Rz q[0] 9.42477796076938
	Rz q[0] 1.5707963267948966
	R q[0] 1.5707963267948966 1.5707963267948966
	Rz q[0] -1.5707963267948966
	Rz q[0] -9.76363515397281e-05
	Rz q[0] 1.5707963267948966
	R q[0] 1.5707963267948966 -1.5707963267948966
	Rz q[0] -1.5707963267948966
	Rz q[0] 1.5707963267948966
	R q[0] 1.5707963267948966 1.5707963267948966
	Rz q[0] -1.5707963267948966
	Sy q[0]
	Rz q[1] 1.5707963267948966
	R q[1] 1.5707963267948966 1.5707963267948966
	Rz q[1] -1.5707963267948966
	Rz q[1] -3.0360625542860462e-05
	Rz q[1] 1.5707963267948966
	R q[1] 1.5707963267948966 -1.5707963267948966
	Rz q[1] -1.5707963267948966
	Pz q[1]
	Sy q[1]
	MS q[0] q[1] 0.0 1.5707963267948966
	Sxd q[0]
	Syd q[0]
	Sy q[0]
	Sxd q[1]
	Rz q[1] 0.07605047960428521
	MS q[0] q[1] 0.0 1.5707963267948966
	Sxd q[0]
	Syd q[0]
	Rz q[0] 1.5707963267948966
	R q[0] 1.5707963267948966 -1.570796

### Scheduling and Emulation

In [16]:
scheduled_jaqal_circuit = schedule_circuit(jaqal_circuit)
print(generate_jaqal_program(scheduled_jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	<
		Rz q[0] 3.141592653589793
		Rz q[1] 1.5707963267948966
	>
	<
		Sx q[0]
		R q[1] 1.5707963267948966 1.5707963267948966
	>
	<
		Rz q[0] 6.283185307179586
		Rz q[1] -1.5707963267948966
	>
	<
		Sx q[0]
		Rz q[1] -3.0360625542860462e-05
	>
	<
		Rz q[0] 9.42477796076938
		Rz q[1] 1.5707963267948966
	>
	<
		Rz q[0] 1.5707963267948966
		R q[1] 1.5707963267948966 -1.5707963267948966
	>
	<
		R q[0] 1.5707963267948966 1.5707963267948966
		Rz q[1] -1.5707963267948966
	>
	<
		Rz q[0] -1.5707963267948966
		Pz q[1]
	>
	<
		Rz q[0] -9.76363515397281e-05
		Sy q[1]
	>
	Rz q[0] 1.5707963267948966
	R q[0] 1.5707963267948966 -1.5707963267948966
	Rz q[0] -1.5707963267948966
	Rz q[0] 1.5707963267948966
	R q[0] 1.5707963267948966 1.5707963267948966
	Rz q[0] -1.5707963267948966
	Sy q[0]
	MS q[0] q[1] 0.0 1.5707963267948966
	<
		Sxd q[0]
		Sxd q[1]
	>
	<
		Syd q[0]
		Rz q[1] 0.07605047960428521
	>
	Sy q[0]
	MS q[0] q[1] 0.0 1.5707963267948

In [17]:
run_jaqal_circuit(scheduled_jaqal_circuit).subcircuits[0].probability_by_str

OrderedDict([('00', 2.383214283050551e-09),
             ('10', 0.9942274635465894),
             ('01', 0.005772533839753675),
             ('11', 2.3044189526917106e-10)])

## Optimizing Circuits for Ion Hardware with t|ket>

In [18]:
from jaqalpaq.transpilers.tket import jaqal_circuit_from_tket_circuit
from pytket.predicates import CompilationUnit
from pytket.extensions.qiskit import qiskit_to_tk
from pytket.passes import SynthesiseUMD, DecomposeBoxes

In [19]:
tket_circuit = qiskit_to_tk(superconducting_circuit)
tket_circuit

[U3(1, 0, 1) q[0]; CircBox q[0], q[1]; CircBox q[0], q[1]; CircBox q[0], q[1]; ]

In [20]:
unit = CompilationUnit(tket_circuit)
DecomposeBoxes().apply(unit)
SynthesiseUMD().apply(unit)
tket_jaqal_circuit = jaqal_circuit_from_tket_circuit(unit.circuit)
print(generate_jaqal_program(tket_jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	Rz q[0] 1.5707963267948966
	Rz q[1] 7.853951273348942
	R q[0] 3.141592653589793 6.283282943531128
	R q[1] 1.5707659661693538 1.5707963267948966
	MS q[0] q[1] 0 1.5707963267948966
	R q[0] 3.141592653589793 1.5707963267948966
	Rz q[1] 0.0760504796042852
	R q[1] 0.0 4.71238898038469
	MS q[0] q[1] 0 1.5707963267948966
	R q[0] 1.5707963267948966 1.5707963267948966
	Rz q[1] 3.141592653589793
	R q[1] 1.5707963267948966 7.853981633974483
	MS q[0] q[1] 0 1.5707963267948966
	R q[0] 3.141592653589793 1.5707963267948966
	Rz q[1] 12.490320134754887
	R q[1] 0.0 4.71238898038469
	MS q[0] q[1] 0 1.5707963267948966
	R q[1] 0.0 10.995574287564276
	measure_all
}



In [21]:
optimized_jaqal_circuit = schedule_circuit(tket_jaqal_circuit)
print(generate_jaqal_program(optimized_jaqal_circuit))

register baseregister[2]

map q baseregister[0:2:1]

{
	prepare_all
	<
		Rz q[0] 1.5707963267948966
		Rz q[1] 7.853951273348942
	>
	<
		R q[0] 3.141592653589793 6.283282943531128
		R q[1] 1.5707659661693538 1.5707963267948966
	>
	MS q[0] q[1] 0 1.5707963267948966
	<
		R q[0] 3.141592653589793 1.5707963267948966
		Rz q[1] 0.0760504796042852
	>
	R q[1] 0.0 4.71238898038469
	MS q[0] q[1] 0 1.5707963267948966
	<
		R q[0] 1.5707963267948966 1.5707963267948966
		Rz q[1] 3.141592653589793
	>
	R q[1] 1.5707963267948966 7.853981633974483
	MS q[0] q[1] 0 1.5707963267948966
	<
		R q[0] 3.141592653589793 1.5707963267948966
		Rz q[1] 12.490320134754887
	>
	R q[1] 0.0 4.71238898038469
	MS q[0] q[1] 0 1.5707963267948966
	R q[1] 0.0 10.995574287564276
	measure_all
}



In [22]:
run_jaqal_circuit(optimized_jaqal_circuit).subcircuits[0].probability_by_str

OrderedDict([('00', 2.3832142831344857e-09),
             ('10', 0.9942274635465896),
             ('01', 0.005772533839753751),
             ('11', 2.3044189527521426e-10)])